In [11]:
%load_ext autoreload
%autoreload 2
import os
import typing as T
import pickle as pkl
import matplotlib.pyplot as plt
import sys
import cv2
import yaml
import requests
import torch
import scipy
import torch.nn as nn
import numpy as np
from xarray import DataArray
from tqdm.auto import tqdm
import io
from PIL import Image
from bioimageio_chatbot.image_processor import *
import numpy as np
from scipy.spatial.distance import cosine
from random import choice

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cellpose_train_images

[]

In [5]:
cellpose_train_dir = "/Users/gkreder/Downloads/train"
cellpose_train_images = [os.path.join(cellpose_train_dir, x) for x in os.listdir(cellpose_train_dir) if x.endswith("_img.png")]
representative_images = get_representative_images(cellpose_train_images, verbose = True)

100%|██████████| 540/540 [00:41<00:00, 13.11it/s]


Embedded database includes 540 vectors


Safety Counter Progress:   5%|▍         | 2594/54000 [10:53<3:35:57,  3.97it/s]


In [15]:
len(representative_images_old)

77

In [13]:
cellpose_train_dir = "/Users/gkreder/Downloads/train"
cellpose_train_images = [os.path.join(cellpose_train_dir, x) for x in os.listdir(cellpose_train_dir) if x.endswith("_img.png")]
representative_images_new = draft_get_representative_images(cellpose_train_images, verbose = True)

Embedding input images


100%|██████████| 540/540 [00:38<00:00, 13.90it/s]

Calculating distance matrix



100%|██████████| 540/540 [00:04<00:00, 108.71it/s]


Assembling representative set


100%|██████████| 540/540 [00:00<00:00, 3164.69it/s]


In [14]:
len(representative_images_new)

541